# Python API Challenge: Vacation destinations map

In [1]:
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey

In [2]:
# Import weather data csv
weather_df = pd.read_csv("../WeatherPy/WeatherData.csv", index_col = False)
weather_df = weather_df.drop(columns = "Unnamed: 0")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tilichiki,60.4667,166.1000,56.48,83,100,1.32,RU,1626294034
1,conceicao do araguaia,-8.2578,-49.2647,94.19,22,1,2.98,BR,1626294034
2,sorong,-0.8833,131.2500,78.64,85,100,6.17,ID,1626293747
3,macau,22.2006,113.5461,87.08,79,20,8.05,MO,1626294035
4,portales,34.1862,-103.3344,88.11,28,8,16.33,US,1626294035


In [3]:
# Search for places that had temperatures between 75 and 80 F, winds below 10mph, and no clouds
vacation_df = weather_df.loc[weather_df["Max Temp"] > 75]
vacation_df = vacation_df.loc[vacation_df["Max Temp"] < 80]
vacation_df = vacation_df.loc[vacation_df["Wind Speed"] < 10]
vacation_df = vacation_df.loc[vacation_df["Cloudiness"] == 0]

vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,letnyaya stavka,45.4306,43.4472,76.21,30,0,3.83,RU,1626294037
17,novopokrovka,49.8425,36.5477,75.58,52,0,0.94,UA,1626294202
159,fez,34.0372,-4.9998,79.05,44,0,3.44,MA,1626302942
258,abu samrah,35.3029,37.1841,76.32,65,0,9.35,SY,1626303165
285,adeje,28.1227,-16.7260,75.22,73,0,2.30,ES,1626303706
302,pontes e lacerda,-15.2261,-59.3353,78.21,36,0,2.77,BR,1626303711
483,can,40.0333,27.0524,75.88,97,0,3.89,TR,1626304225


In [6]:
# Heat map of locations in WeatherData.csv, weighted by humidity
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Search for hotels within 5000m of each city using Google Places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Add a column to store the name of the nearest hotel
hotel_df = vacation_df.copy()
hotel_df["Hotel Name"] = ""

params = {
    "rankby": "distance",
    "keyword": "hotel",
    "key": gkey
}

for index, row in vacation_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    try:
        response = requests.get(base_url, params = params).json()
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = 0

In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
13,letnyaya stavka,45.4306,43.4472,76.21,30,0,3.83,RU,1626294037,"Gostinitsa ""Grand Otel'"""
17,novopokrovka,49.8425,36.5477,75.58,52,0,0.94,UA,1626294202,Hotel Meduza
159,fez,34.0372,-4.9998,79.05,44,0,3.44,MA,1626302942,Splendid Hôtel
258,abu samrah,35.3029,37.1841,76.32,65,0,9.35,SY,1626303165,Borj hama
285,adeje,28.1227,-16.7260,75.22,73,0,2.30,ES,1626303706,Royal River Luxury Hotel
302,pontes e lacerda,-15.2261,-59.3353,78.21,36,0,2.77,BR,1626303711,Plaza Hotel
483,can,40.0333,27.0524,75.88,97,0,3.89,TR,1626304225,Kardeş Hotel


In [9]:
# Add clickable markers to the heatmap at hotel locations
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations = locations,
                  info_box_content = hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))